In [1]:
import pandas as pd
import numpy as np

queries = ('q11', 'q12', 'q13', 'q21', 'q22', 'q23', 'q31', 'q32', 'q33', 'q34', 'q41', 'q42', 'q43')

def load_performance_data(query):
    return pd.read_csv(f'{query}-performance.log', header=None, names=['execution_time_ms'])

In [4]:
results = pd.DataFrame()
for q in queries:
    data = load_performance_data(q)
    results[q] = data['execution_time_ms']
    
results.describe()

,q11,q12,q13,q21,q22,q23,q31,q32,q33,q34,q41,q42,q43
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,13.068655,14.049552,13.996908,14.179259,10.679986,9.148562,16.849799,82.012622,80.656472,91.153376,19.731310,18.112664,431.043600
std,0.086559,0.115830,0.144276,1.107210,1.032482,1.149088,1.237405,5.086053,4.573994,4.363834,1.402337,1.494186,7.936452
min,12.992100,13.474100,13.911000,13.186100,9.808070,8.219010,15.652100,77.429500,75.664400,86.606400,17.640500,15.734100,420.843000
25%,13.032450,14.020050,13.957275,13.380750,9.921978,8.356628,15.890900,79.765375,78.651475,89.064225,18.926775,17.168500,426.222500
50%,13.050350,14.032050,13.969500,13.515050,10.030550,8.434475,16.001600,81.078450,79.647650,90.464500,19.102100,17.433850,429.345500
75%,13.076600,14.051125,13.988625,15.053700,11.473100,10.019300,17.937850,82.807375,81.133775,91.962500,20.856650,19.449800,433.334250
max,13.671200,14.993700,15.345900,17.472100,13.998400,14.118000,20.316200,122.785000,122.008000,129.957000,23.520500,21.488900,484.644000


In [5]:
results.head(100).describe()

,q11,q12,q13,q21,q22,q23,q31,q32,q33,q34,q41,q42,q43
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,13.065509,14.043181,13.978978,13.931243,10.498814,9.004495,16.591754,81.549913,80.058497,91.078296,19.395202,17.698262,429.310890
std,0.097738,0.127178,0.084759,0.793319,0.742388,0.879367,0.957681,5.026561,3.520723,5.426912,1.163444,1.131839,6.181581
min,12.992100,13.474100,13.911000,13.186100,9.808070,8.219010,15.660700,77.429500,75.664400,86.606400,17.640500,15.734100,420.843000
25%,13.022500,14.016175,13.944775,13.352100,9.906095,8.339840,15.855800,79.474475,78.517175,89.045625,18.876775,17.127925,424.871500
50%,13.043300,14.028100,13.963100,13.489100,10.023450,8.431110,15.973850,80.842050,79.322250,90.248850,18.988100,17.365950,428.108000
75%,13.069500,14.043000,13.976525,14.631850,11.236275,9.724490,17.341900,82.235075,80.355200,91.247250,20.375750,18.530025,431.779750
max,13.671200,14.769700,14.466400,16.329900,12.431100,12.518200,20.316200,118.289000,99.198100,129.957000,22.970200,19.864200,464.485000


In [ ]:
results[100:].describe()